In [1]:
import numpy as np

days2sec = 86400  # days to seconds conversion factor
sec2yr = 3.1688087814029e-8  # seconds to year conversion factor
r_e = 6371e3  # Earth radius [m]
m_e = 5.972e24  # Earth mass [kg]
G = 6.674e-11  # gravitational constant [si]

# Default parameters - not affecting the outgassing rate
cmf = 0.01  #0.358  # metal core mass fraction
rho_l = 2800  # magma density [kg/m^3]
dH_f = 4.5e5  # latent heat of fusion of silicate [J/kg]
cp = 1000  # specific heat capacity of magma [J/kg/K]
T_l = 1500  # magma temperature [K]

# Recommended maximum values for heating rate and volatile contents
q_sfc_darcy = 4e3  # surface heat flux from Darcy segregation [W/m^2]
w_c = 2e-2  # melt carbon capacity
w_s = 2e-2  # melt sulfur capacity

In [2]:
# Main model

def surface_area(r_p=None):
    """
    
    """
    return 4 * np.pi * r_p ** 2  # planet surface area [m^2]
                 
                 
def surface_gravity(r_p, m_p):
    """
    
    """
    return G * m_p / r_p ** 2  # surface gravity [m/s^2]
                 

def planet_equilibrium_temperature(s=1, L_star=None, r_star=None, T_star=5778, albedo=0.3, **kwargs):
    """
    
    """
    d = s * 1.496e11  # semi-major axis, convert AU [m]
    if L_star is None:
        # Calculate from radius and effective temperature
        assert r_star and T_star, 'Need L_star or both r_star and T_star to calculate planetary equilibrium temperature.'
        r_s = r_star * 696340e3  # star radius, convert solar radii [m]
        T_eq = T_star * (1 - albedo) ** 0.25 * np.sqrt(r_s / (2 * d))  # planet equilibrium temperature [K]
    else:
        L = L_star * 3828e26  # convert to W
        sigma = 5.670374419e-8  # Stefan-Boltzmann constant [W/m^2/K^4]
        T_eq = (L * (1 - albedo)/(16 *sigma * np.pi * s**2)) ** (1/4)
    return T_eq
                 
                 
def quick_tidal_heating(r_p, period, ecc, imag_k2_min=-0.3):
    """
    
    """
    freq = 2 * np.pi / (period * days2sec)
    power = -21/2 * imag_k2_min * (freq * r_p) ** 5 / G * ecc ** 2  # [W]
    return power  # [W]


def quick_radiogenic_heating(age, m_mantle, x_rprocess=3):
    """
    
    """
    # Radioisotope defaults using O'Neill+ 2020 SSR equation (1) - [40K, 238U, 235U, 232Th]
    tau_i = np.array([1250, 4468, 703.8, 14050])  # half life in Myr
    h_i = np.array([28.761e-6, 94.946e-6, 568.402e-6, 26.368e-6])  # heat production in W/kg
    c_i = np.array([30.4e-9, 22.7e-9, 0.16e-9, 85e-9])  # BSE concentration in kg/kg
    
    age_Myr = age * 1e3
    h_K = np.array(c_i[0] * h_i[0] * np.exp((4.5e3 - age_Myr) * np.log(2) / tau_i[0]))  # potassium
    h_UTh = np.array(sum(c_i[1:] * h_i[1:] * np.exp((4.5e3 - age_Myr) * np.log(2) / tau_i[1:])))  # U and Th
    return (h_K + x_rprocess * h_UTh) * m_mantle  # [W]
                 
                 
def melt_mass_flux(q_s=None, sa_p=None, dH_f=None, cp=None, T_l=None, T_s=None, f_a=1, verbose=False, **kwargs):
    """
    
    """
    q_a = f_a * q_s  # advective heat flux [W m^-2]
    f_melt = q_a * sa_p / (dH_f + cp * (T_l - T_s))  # rate of magma supply to the surface in kg/s
    return f_melt


def min_initial_volatiles(f_vol, m_mantle, age):
    """
    
    """
    f_vol_cumulative = f_vol * age * 1e9 / sec2yr  # [kg]
    mass_frac_volatiles = f_vol_cumulative / m_mantle
    return mass_frac_volatiles

STEP 0: Enter planet parameters

In [25]:
L 98-59 b

r_p = 0.837  # planet radius [R_Earth]
m_p = 0.46  # planet mass [M_Earth]
age = 4.94  # system age [Gyr]
period = 2.271  # orbital period [days]
ecc = 0.167  # orbital eccentricity
sma = 0.0223  # semi-major axis [AU]
r_star = 0.3155  # stellar radius [R_Sun]
T_star = 3340  # stellar effective temperature [K]

sa_p = surface_area(r_p * r_e)
g_s = surface_gravity(r_p * r_e, m_p * m_e)
T_s = planet_equilibrium_temperature(sma, r_star=r_star, T_star=T_star)

# LTT 1445A b

# r_p = 1.18  # planet radius [R_Earth]
# m_p = (r_p * r_e * 1e-3 / (7030 - 1840 * cmf)) ** (1/0.282)  # planet mass [M_Earth]
# age = 5 # system age [Gyr] - UNKNOWN
# period = 5.35876  # orbital period [days]
# ecc = 0.19  # orbital eccentricity
# sma = 0.022  # semi-major axis [AU]
# r_star = 0.271  # stellar radius [R_Sun]
# T_star = 3415  # stellar effective temperature [K]

# # TOI 431 b

# period = 0.490047
# r_p = 1.28
# m_p = 3.07
# age = 5  # unknown
# ecc = 0
# sma = 0.0113
# T_star = 4850
# r_star = 0.731

# # LHS 1478 b

# period = 1.9495378
# r_p = 1.242
# m_p = 2.33
# age = 5  # unknown
# ecc = 0
# sma = 0.01848
# T_star = 3381
# r_star = 0.246

# sa_p = surface_area(r_p * r_e)
# g_s = surface_gravity(r_p * r_e, m_p * m_e)
# T_s = planet_equilibrium_temperature(sma, r_star=r_star, T_star=T_star)
# m_mantle = m_p * m_e * (1 - cmf)

STEP 1: Can tidal heating be noticeable?

In [26]:
# Estimate surface heat flux due to tidal heating
q_s_tides = quick_tidal_heating(r_p * r_e, period, ecc, imag_k2_min=-0.3) / surface_area(r_p * r_e)  # [W/m^2]
q_s_rad = quick_radiogenic_heating(age, m_mantle) / sa_p  # [W/m^2]

if q_s_tides > q_s_rad:
    # Limit heating rate to <40% melt fraction
    q_s_max = np.minimum(q_s_tides, q_sfc_darcy)
    print('Strong tidal heating possible = {:.2e} W/m^2'.format(q_s_max))
    
else:
    q_s_max = q_s_rad
    print('Using maximum radiogenic heating = {:.2e} W/m^2'.format(q_s_rad))

Using maximum radiogenic heating = 7.42e-02 W/m^2


STEP 2: Estimate maximum outgassing flux

In [27]:
f_melt_max = melt_mass_flux(q_s_max, sa_p, dH_f, cp, T_l, T_s)
f_carbon = f_melt_max * w_c
f_sulfur = f_melt_max * w_s

print('Maximum carbon outgassing = {:.2e} kg/s'.format(f_carbon))
print('Maximum sulfur outgassing = {:.2e} kg/s'.format(f_sulfur))
print('Maximum resurfacing rate = {:.2e} m/s = {:.2e} km^3/yr'.format(f_melt_max / sa_p / rho_l, 
                                                                      f_melt_max / rho_l / sec2yr * 1e-9))

Maximum carbon outgassing = 8.30e+05 kg/s
Maximum sulfur outgassing = 8.30e+05 kg/s
Maximum resurfacing rate = 1.88e-11 m/s = 4.68e+02 km^3/yr


STEP 3: Check supply limit

In [21]:
s_ini = min_initial_volatiles(f_sulfur, m_p * m_e * (1 - cmf), age)

print('Minimum initial S in bulk silicate planet = {:.3f}%'.format(s_ini * 100))

Minimum initial S in bulk silicate planet = 0.414%
